# Introdução

Neste caderno, examinaremos a definição, o treinamento e os testes, bem como as etapas de pré-processamento necessárias para alimentar uma imagem nas CNNs para obter resultados de última geração. Usaremos o PyTorch para este tutorial. O PyTorch é uma poderosa estrutura de aprendizado profundo que está crescendo em popularidade e é completamente à vontade no Python, o que facilita muito o aprendizado e o uso. Este tutorial não assume muito em relação ao conhecimento prévio do PyTorch, mas pode ser útil fazer o checkout [meu tutorial introdutório anterior de CV] (https://www.kaggle.com/abhinand05/mnist-introduction-to-computervision- com pytorch).


![FeaturedImage](https://i.ibb.co/ws3htpn/2088474-6a86-3.jpg)



Neste caderno, treinaremos a CNN para classificar imagens com base no fato de terem ou não um cacto colunar. Usaremos o Conjunto de dados aéreos dos cactos [desta competição atualmente em execução no Kaggle] (https://www.kaggle.com/c/aerial-cactus-identification/overview). Para obter mais informações sobre o conjunto de dados, visite [esta página] (https://www.kaggle.com/c/aerial-cactus-identification/data). Eu escolhi essa competição porque achava que é o melhor lugar para os iniciantes praticarem suas novas habilidades encontradas com as CNNs, já que o MNIST é muito simples para colocar as CNNs em jogo. Um perceptron regular de várias camadas pode muito bem fazer o trabalho. Portanto, esta é uma competição perfeita para iniciantes, como alguém disse corretamente nos fóruns de discussão.

### ** Se você gosta deste kernel ou deseja bifurcá-lo, por favor, faça um UPVOTE para mostrar sua apreciação. **

# Importando bibliotecas

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir('../input/'))

import matplotlib.pyplot as plt
plt.style.use("ggplot")

# OpenCV Image Library
import cv2

# Import PyTorch
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision
import torch.optim as optim

## Carregamento de dados de treinamento + EDA

In [ ]:
train_df = pd.read_csv("../input/aerial-cactus-identification/train.csv")
train_df.head()

Os dados do trem contêm 17500 imagens que podem ser encontradas em um diretório separado e também temos um arquivo csv, mas não podemos visualizá-las diretamente, como veremos mais adiante.

Os dados de teste têm 4000 imagens e são armazenados em um diretório separado. Observe que ele não possui um arquivo CSV como vimos para os dados do traino

In [ ]:
print(f"Train Size: {len(os.listdir('../input/aerial-cactus-identification/train/train'))}")
print(f"Test Size: {len(os.listdir('../input/aerial-cactus-identification/test/test'))}")

Aqui, inspecionamos os dados para ver a quantidade de amostras de treinamento para cada classe. Como podemos ver, cerca de 2/3 dos dados de treinamento pertencem a uma classe. Se o conjunto de dados que você estiver usando contiver quase ou acima de 90% dos dados de treinamento pertencentes a uma única classe, isso afetará bastante seus resultados.
Isso é chamado de classes distorcidas, podemos usar aumento de dados, amostragem e várias maneiras de superar isso. No entanto, não acredito que seja um problema aqui, pois temos dados suficientes para que as CNNs obtenham ótimos resultados.

In [ ]:
# Contando o número de dados de amostra para cada classe
value_counts = train_df.has_cactus.value_counts()
%matplotlib inline
plt.pie(value_counts, labels=['Has Cactus', 'No Cactus'], autopct='%1.1f', colors=['green', 'red'], shadow=True)
plt.figure(figsize=(5,5))
plt.show()

## Configurando dados de trem para Pytorch
Não podemos simplesmente usar os dados brutos da imagem para fazer previsões usando o PyTorch. Existem várias etapas de pré-processamento envolvidas que discutiremos em detalhes nesta seção.

**Passo 1:**

Primeiro, definimos uma classe personalizada que estende a classe `torch.utils.data.Dataset` do PyTorch. Eu acho que tudo lá é bem direto. Definimos nossos construtores e adicionamos dois métodos diferentes, `len` e` getitem`, que substituem essencialmente as definições dos pais.

In [ ]:
# Caminhos de dados
train_path = '../input/aerial-cactus-identification/train/train/'
test_path = '../input/aerial-cactus-identification/test/test/'

In [ ]:
# Nossa própria classe personalizada para conjuntos de dados
class CreateDataset(Dataset):
    def __init__(self, df_data, data_dir = './', transform=None):
        super().__init__()
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_name,label = self.df[index]
        img_path = os.path.join(self.data_dir, img_name)
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

**Passo 2:**

Agora que definimos nossa classe, é hora de passar os dados brutos e convertê-los para o formato compreensível do PyTorch.

** Transformações ** - podemos usar o recurso de transformações no PyTorch para aplicar aumentos de dados que nos ajudam a melhorar a precisão do nosso modelo quando bem feito. Existem transformações sevarais que podem ser aplicadas e você pode dar uma olhada na [documentação aqui] (https://pytorch.org/docs/stable/torchvision/transforms.html). Aqui, nós a inserimos primeiro em uma imagem PIL. A inversão horizontal aleatória de imagens de amostra é aplicada juntamente com rotação aleatória de 10 graus para exemplos de treinamento aleatório. Em seguida, convertemos as imagens em um tensor PyTorch e normalizamos as imagens.

** Criando nosso conjunto de dados - ** Em seguida, usamos nossa classe `CreateDataset` para ocultar os dados brutos da maneira que o PyTorch espera. Também aplicamos as transformações lá.

Aplicando o Data Augmentation

In [ ]:
transforms_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = CreateDataset(df_data=train_df, data_dir=train_path, transform=transforms_train)

**Etapa 3:**

O tamanho do lote está definido. O tamanho do lote geralmente é definido entre 64 e 256. O tamanho do lote afeta a precisão do teste final. Uma maneira de pensar sobre isso é que lotes menores significam que o número de atualizações de parâmetros por época é maior.

Em seguida, a porcentagem de dados necessários para a validação é definida em 20%, o que quase sempre parece funcionar para mim. Mas no final, é apenas outro hiperparâmetro que você pode ajustar.

Nas próximas etapas, usamos a função `torch.utils.data.samplerSubsetRandomSampler` para dividir nossos dados em conjuntos de treinamento e validação, que é semelhante à função` train_test_split` do scikit-learn.

Temos os dados de treinamento passados para o carregador de trem. Podemos fazer um iterador com o iter (trainloader) que pode nos ajudar a coletar dados. Mais tarde, usaremos isso para percorrer o conjunto de dados para treinamento. Cada vez que podemos extrair dados do tamanho do lote definido.

In [ ]:
# Definir tamanho do lote
batch_size = 64

# Porcentagem de treinamento definida para uso como validação
valid_size = 0.2

# obter índices de treinamento que serão usados para validação
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# Criar Samplers
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# preparar carregadores de dados (combinar conjunto de dados e amostrador)
train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
valid_loader = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)

## Configurando dados de teste
Esta seção do código deve fazer sentido para você agora. As mesmas etapas usadas acima são repetidas para os dados de teste também.

Observe que não aumentamos nossos dados no conjunto de treinamento. Isso ocorre porque o aprimoramento é feito apenas no conjunto de treinamento para melhorar o desempenho, fornecendo ao nosso modelo variações complexas que podem torná-lo generalizado para novas amostras no conjunto de testes, portanto, não faz sentido fazer o aprimoramento de dados nos dados de teste também. No entanto, ainda precisamos convertê-lo em um tensor e normalizá-lo.

In [ ]:
transforms_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# criando dados de teste
sample_sub = pd.read_csv("../input/aerial-cactus-identification/sample_submission.csv")
test_data = CreateDataset(df_data=sample_sub, data_dir=test_path, transform=transforms_test)

# prepare o carregador de teste
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

## Visualizar imagens

Visualizar as imagens e inspecioná-las para ter uma idéia melhor do que são são sempre úteis antes de construir o modelo para fazer previsões.

Podemos ver imagens aéreas de cactos. As imagens contêm canais de cores, devemos ter em mente e são imagens de 32x32. Observe que essas imagens são de baixa resolução e, como seres humanos, evoluímos de maneira a dar sentido a esse tipo de imagem. Vamos construir uma CNN nas próximas seções para, no final das contas, alcançar exatamente esse ou um desempenho ainda melhor.

In [ ]:
classes = [ 'No Cactus','Cactus']

In [ ]:
def imshow(img):
    '''Helper function to un-normalize and display an image'''
    # anormalizar
    img = img / 2 + 0.5
    # converter da imagem Tensor e exibir
    plt.imshow(np.transpose(img, (1, 2, 0)))

In [ ]:
# obter um lote de imagens de treinamento
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # converter imagens em numpy para exibição

# plota as imagens no lote, juntamente com os rótulos correspondentes
fig = plt.figure(figsize=(25, 4))
# exibir 20 imagens
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])

## Visualizando uma imagem com mais detalhes
Aqui, examinamos os canais de cores normalizados de vermelho, verde e azul (RGB) como três imagens separadas de intensidade de escala de cinza para fins de ilustração. Isso me ajuda a explicar as CNNs mais tarde.

Cada pequeno quadrado que você pode ver são pixels com seus valores normalizados. Valor maior significa mais brilhante, mais baixo significa mais escuro. É assim que as imagens são representadas em nossos computadores. Quando combinamos os três canais, obtemos uma imagem colorida. Esse esquema de cores RGB pode representar cerca de 16,77 milhões de cores, o que é incrível.

Como você já deve ter imaginado, as imagens coloridas são na verdade tridimensionais, onde as imagens em escala de cinza, por exemplo, são 1D.

In [ ]:
rgb_img = np.squeeze(images[3])
channels = ['red channel', 'green channel', 'blue channel']

fig = plt.figure(figsize = (36, 36)) 
for idx in np.arange(rgb_img.shape[0]):
    ax = fig.add_subplot(3, 1, idx + 1)
    img = rgb_img[idx]
    ax.imshow(img, cmap='gray')
    ax.set_title(channels[idx])
    width, height = img.shape
    thresh = img.max()/2.5
    for x in range(width):
        for y in range(height):
            val = round(img[x][y],2) if img[x][y] !=0 else 0
            ax.annotate(str(val), xy=(y,x),
                    horizontalalignment='center',
                    verticalalignment='center', size=8,
                    color='white' if img[x][y]<thresh else 'black')

## Perceptrons de várias camadas (MLP) versus redes neurais convolucionais (CNN)
Como você deve ter visto no [meu tutorial anterior] (https://www.kaggle.com/abhinand05/mnist-introduction-to-computervision-with-pytorch) no conjunto de dados MNIST, os MLPs foram bons o suficiente para marcar mais de 90% precisão. Acontece que eles não são nem bons o suficiente para obter resultados semelhantes em conjuntos de dados complexos, principalmente porque esperam um vetor achatado da imagem original como o abaixo. O conjunto de dados MNIST é uma exceção, pois já vem bem processado.

![Example](https://github.com/abhinand5/CNNs-in-PyTorch/blob/master/cifar-cnn/FireShot%20Capture%20084%20-%20Intro%20to%20Deep%20Learning%20with%20PyTorch%20-%20Udacity%20-%20classroom.udacity.com.png?raw=true)

Então, tudo o que vê é um vetor e o trata apenas como um vetor sem estrutura especial. Não tem conhecimento algum do fato de ter sido previamente organizado em uma grade, perdendo informações importantes. Se fazemos o mesmo com imagens coloridas, descartamos informações vitais, achatando-as, o que quase nunca funciona para imagens do mundo real. Por outro lado, as CNNs funcionam exatamente do contrário, capazes de trabalhar e elucidar padrões a partir de dados multidimensionais, por isso são tão poderosos. Diferentemente dos MLPs, as CNNs realmente entendem as informações dos pixels que estão próximos um do outro e mais relacionados entre si do que os pixels que estão distantes e não relacionados.


## Noções básicas sobre CNNs

As Redes Neurais Conolucionais usam três classes diferentes de camadas que diferem distintamente entre si. Mas quando combinados, eles dão resultados extraordinários.

* Camadas Convolucionais
* Camadas de pool
* Camadas totalmente conectadas

Analisaremos cada uma das camadas em grande detalhe.

Vou usar um exemplo de classificação de um carro aqui para explicar as diferentes camadas de uma Rede Neural Convolucional.

![Layers](https://s3.amazonaws.com/video.udacity-data.com/topher/2018/May/5b1070e4_screen-shot-2018-05-31-at-2.59.36-pm/screen-shot-2018-05-31-at-2.59.36-pm.png)

** Camada Convolucional: **

Acontece que, para entender primeiro a camada Convolucional, precisamos entender um conceito chamado Filtros. Filtros especialmente passa-alto.

Para detectar alterações na intensidade de uma imagem, usaremos e criaremos filtros de imagem específicos que analisam grupos de pixels e reagem a padrões alternados de pixels escuros / claros. Esses filtros produzem uma saída que mostra bordas dos objetos e texturas diferentes.

Para isso, primeiro construímos um filtro de acordo com um tamanho chamado tamanho da janela, que nada mais é do que uma matriz como essa.

Aqui em um exemplo gif.

![Example](https://media.giphy.com/media/jrzu0JxxZydz0valeu/giphy.gif)

Essas janelas são geralmente do tamanho 3x3, o que ajuda as CNNs a identificar os padrões em uma imagem. Esses filtros podem ser modificados para obter filtros diferentes como saída. Pode haver vários filtros em uma camada convolucional.

A camada convolucional é produzida aplicando uma série de muitos filtros de imagem diferentes, também conhecidos como kernels convolucionais, a uma imagem de entrada.

![Example](https://s3.amazonaws.com/video.udacity-data.com/topher/2018/May/5b10723a_screen-shot-2018-05-31-at-3.06.07-pm/screen-shot-2018-05-31-at-3.06.07-pm.png)

No exemplo mostrado, 4 filtros diferentes produzem 4 imagens de saída filtradas de maneira diferente. Quando empilhamos essas imagens, formamos uma camada convolucional completa com profundidade de 4!

![Example](https://s3.amazonaws.com/video.udacity-data.com/topher/2018/May/5b10729b_screen-shot-2018-05-31-at-3.07.03-pm/screen-shot-2018-05-31-at-3.07.03-pm.png)

A profundidade de cada camada convolucional consecutiva pode aumentar, resultando na captura de redes de características / padrões incrivelmente complexos das imagens. De fato, isso é muito parecido com o modo como nossos cérebros interpretam imagens em um instante.

** Camada de pool: **

O pool é usado em redes neurais convolucionais para tornar a detecção de certos recursos um tanto invariável às mudanças de escala e orientação. Outra maneira de pensar sobre o que o pool faz é que ele generaliza em informações mais complexas e de nível inferior.

Como isso é feito é uma janela de tamanho fixo feita para passear sobre a imagem e, dentro da janela, um valor específico de acordo com uma métrica é calculado e um novo tensor é formado. Quando tiramos o elemento máximo da janela, ele é conhecido como Max-Pooling, que é a técnica de pool mais comum. Também existe um pool médio que você pode ver na natureza. Aqui está uma ilustração que você pode achar útil. Existem também alternativas ao Pooling, como a Capsule Networks, que está fora da pontuação deste kernel.

Aqui em um exemplo de gif de Max Pooling.

![Example](https://media.giphy.com/media/U7PsR7cv9oIcB6eEAd/giphy.gif)

Quando combinamos camadas de pool com camadas convolucionais, reduzimos a dimensão das camadas, o que ajuda na computação, mas mais do que isso seleciona os valores de pixel de maior significado. Até agora, nosso modelo se parece com isso ...

![Example](https://i.ibb.co/V06mcY0/Intro-to-Deep-Learning-with-Py-Torch-Udacity-classroom-udacity-com.png)

** Camadas totalmente conectadas: **

As camadas totalmente conectadas não são diferentes daquelas que você já conhece dos MLPs. É a peça final do quebra-cabeça que o torna especial e poderoso. Uma forma ainda mais refinada é passada para as camadas FC para fazer a previsão final.

Aí vem o nosso modelo final ...

![Example](http://cs231n.github.io/assets/cnn/convnet.jpeg)


Eu sei que essa pode não ser a melhor das definições que você veria para as CNNs, mas o objetivo aqui é torná-la pelo menos vagamente compreensível. Há muito mais acontecendo nos bastidores que eu pulei. Encorajo-vos a sair e explorar por conta própria para encontrar essas coisas impressionantes. Espero que isso faça sentido para você. Vamos continuar.

### ** Se você gosta deste kernel ou deseja bifurcá-lo, por favor, faça um UPVOTE para mostrar sua apreciação. **

Créditos da imagem: Stanford CS231n, Udacity

## Definir a arquitetura de rede

Aí vem a parte importante da definição de uma CNN que pode ser feita usando o módulo `torch.nn`. Primeiro, você deve definir uma classe Model e preencher duas funções `__init__` e` __forward__`. Agora que você entende como a CNN funciona, tudo é praticamente auto-explicativo. Leia os documentos para `nn.Conv2d` para saber mais sobre os parâmetros.

* Definimos primeiro as camadas convolucionais. Os detalhes podem ser principalmente interpretados a partir das próprias linhas de comentário. Mas deixe-me explicar um. Nossa imagem (RGB) tem 3 canais com profundidade = 3, é por isso que a nossa primeira camada de conv tem 3 canais de entrada. Decidi ter 16 filtros para a primeira camada de conv, então out_channels = 16, pois produz 16 filtros diferentes, o kernel tamanho aka tamanho da janela é definido como 3 com padding = 1, que cria espaços extras nas bordas da imagem para ajudar o kernel 3x3 a deslizar sobre todos os pixels das bordas da imagem em caso de incompatibilidade de tamanho.

* Em seguida, definimos as camadas de pool onde um tamanho de janela e passo de 2 são definidos. Exatamente o mesmo que o exemplo de gif que você viu anteriormente.

* As saídas são então conectadas a uma camada FC.

* O abandono é uma técnica de regularização para evitar ajustes excessivos, que também é adicionada. Tudo é montado em uma função de propagação direta posteriormente.

* Você também pode usar a Normalização em lote, mas decidi não, porque atingi mais de 99% de precisão sem ela.

* As saídas aqui não precisam ser 0 ou 1, o que o Kaggle espera são as probabilidades, portanto não precisamos de uma função de ativação como sigmóide na camada de saída.

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Camada Convolucional (veja o tensor de imagem 32x32x3)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        # Camada convolucional (veja o tensor da imagem 16x16x16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # Camada convolucional (consulte o tensor de imagem 8x8x32)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # Camada Convolucional (veja o tensor de imagem 4 * 4 * 64)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        # Camada Maxpooling
        self.pool = nn.MaxPool2d(2, 2)
        # Camada 1 linear totalmente conectada (consulte o tensor de imagem 2 * 2 * 128)
        self.fc1 = nn.Linear(128*2*2, 512)
        # Camada FC linear 2
        self.fc2 = nn.Linear(512, 2)
        # Definir desistência
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        # adicionar sequência de camadas convolucionais e de max pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        # achatar a entrada da imagem
        x = x.view(-1, 128 * 2 * 2)
        # adicionar camada de desistência
        x = self.dropout(x)
        # adicione a 1ª camada oculta, com função de ativação relu
        x = F.relu(self.fc1(x))
        # adicionar camada de desistência
        x = self.dropout(x)
        # adicione a segunda camada oculta, com função de ativação relu
        x = self.fc2(x)
        return x

In [ ]:
# verifique se CUDA está disponível
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
# crie uma CNN completa
model = CNN()
print(model)

# Mover modelo para GPU, se disponível
if train_on_gpu: model.cuda()

## Treinando nossa CNN

Agora, as etapas de treinamento são as mesmas do treinamento de um MLP, que expliquei em um kernel anterior. A única diferença aqui é que eu estou salvando o modelo toda vez que a perda de validação diminui. Finalmente, obteremos os melhores parâmetros de modelo aprendidos. Este é um tipo de parada antecipada.

In [ ]:
# especificar função de perda (perda de entropia cruzada categórica)
criterion = nn.CrossEntropyLoss()

# especificar otimizador
optimizer = optim.Adamax(model.parameters(), lr=0.001)

In [ ]:
# número de épocas para treinar o modelo
n_epochs = 40

valid_loss_min = np.Inf # rastrear alteração na perda de validação

# acompanhando as perdas assim que acontecem
train_losses = []
valid_losses = []

for epoch in range(1, n_epochs+1):

    # acompanhar as perdas de treinamento e validação
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # treinar o modelo#
    ###################
    model.train()
    for data, target in train_loader:
        # mover tensores para GPU se CUDA estiver disponível
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # limpe os gradientes de todas as variáveis otimizadas
        optimizer.zero_grad()
        # forward forward: calcula as saídas previstas passando entradas para o modelo
        output = model(data)
        # calcular a perda de lote
        loss = criterion(output, target)
        #retrocesso: calcular o gradiente da perda em relação aos parâmetros do modelo
        loss.backward()
        # execute uma única etapa de otimização (atualização de parâmetro)
        optimizer.step()
        # atualizar perda de treinamento
        train_loss += loss.item()*data.size(0)
        
    ######################    
   # validar o modelo #
    ######################
    model.eval()
    for data, target in valid_loader:
        # mova tensores para a GPU se CUDA estiver disponível
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
       # forward pass: calcula as saídas previstas passando entradas para o modelo
        output = model(data)
        # calcular a perda de lote
        loss = criterion(output, target)
       # atualizar perda média de validação
        valid_loss += loss.item()*data.size(0)
    
   # calcular perdas médias
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
        
   # imprimir estatísticas de treinamento / validação
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # salvar modelo se a perda de validação diminuiu
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'best_model.pt')
        valid_loss_min = valid_loss

## Gráfico de desempenho
Eu lhe disse o quão poderosas as CNNs são agora olhem para esse gráfico. Atingimos resultados avançados, como prometido.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.plot(train_losses, label='Training loss')
plt.plot(valid_losses, label='Validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(frameon=False)

In [ ]:
# Carregar Os melhores parâmetros aprendidos com o treinamento em nosso modelo para fazer previsões mais tarde
model.load_state_dict(torch.load('best_model.pt'))

## Fazer previsões no conjunto de testes

In [ ]:
# Desativar gradientes
model.eval()

preds = []
for batch_i, (data, target) in enumerate(test_loader):
    data, target = data.cuda(), target.cuda()
    output = model(data)

    pr = output[:,1].detach().cpu().numpy()
    for i in pr:
        preds.append(i)

# Criar arquivo de envio     
sample_sub['has_cactus'] = preds
sample_sub.to_csv('submission.csv', index=False)

### ** Se você gosta deste kernel ou deseja bifurcá-lo, por favor, faça um UPVOTE para mostrar sua apreciação. **

** Autor: **

[Abhinand](https://www.kaggle.com/abhinand05)